**Importing the dependencies**

In [1]:
import os
import json
import requests

In [2]:
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

In [12]:
# laoding the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]  
workspace_name = data["workspace_name"]
region = data["region"]

In [13]:
print(resource_group)
print(workspace_name)
print(region)

mlsa-model-deploy-raza
deploy-mlsa-model-deploy-raza
centralindia


**Create a Resource Group from Azure Portal**

In [14]:
# Create a workspace

ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

Deploying KeyVault with name deploymlkeyvault2ec36740.
Deploying AppInsights with name deploymlinsights00c84da0.
Deployed AppInsights with name deploymlinsights00c84da0. Took 4.1 seconds.
Deploying StorageAccount with name deploymlstorage7a9cb8d8a.
Deployed KeyVault with name deploymlkeyvault2ec36740. Took 22.35 seconds.
Deployed StorageAccount with name deploymlstorage7a9cb8d8a. Took 29.76 seconds.
Deploying Workspace with name deploy-mlsa-model-deploy-raza.
Deployed Workspace with name deploy-mlsa-model-deploy-raza. Took 34.95 seconds.
Workspace deploy-mlsa-model-deploy-raza created


In [6]:
# Specify the path to your  model file
model_path = 'diabetes_model.pkl'

In [7]:
model_name='diabetes_prediction_model'

In [15]:
# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model diabetes_prediction_model


In [16]:
# Create a Conda environment for your scikit-learn model
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

In [17]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

In [18]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

In [24]:
service = Model.deploy(workspace=ws,
                       name='diabetes-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)
service.wait_for_deployment(show_output=True)

C:\Users\mohdr\AppData\Local\Temp\ipykernel_13592\3861565384.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-06-21 00:04:13+05:30 Creating Container Registry if not exists.
2024-06-21 00:04:17+05:30 Use the existing image.
2024-06-21 00:04:17+05:30 Generating deployment configuration.
2024-06-21 00:04:18+05:30 Submitting deployment to compute.
2024-06-21 00:04:26+05:30 Checking the status of deployment diabetes-prediction-service..
2024-06-21 00:05:29+05:30 Checking the status of inference endpoint diabetes-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [25]:
scoring_uri = service.scoring_uri

In [26]:
scoring_uri

'http://f1c05fa2-3956-4a97-9e3e-64315824bb72.centralindia.azurecontainer.io/score'

**Cleaning up all the created resources**

In [22]:
# Delete the service

service_name = "diabetes-prediction-service"

service = ws.webservices[service_name]
service.delete()

Running
2024-06-20 23:57:52+05:30 Check and wait for operation (9a1c954f-38d5-42ca-b3be-bff757c851d3) to finish.
2024-06-20 23:57:57+05:30 Deleting service entity.
Succeeded


In [23]:
# Specify the name of the registered model to delete
model_name = 'diabetes_prediction_model'

# Delete the model
registered_model = ws.models[model_name]
registered_model.delete()


KeyboardInterrupt: 

In [17]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)

1. Azure Account with appropriate subscription
2. Configuration file that has subscription_id and other details like resource_group name, etc
3. Create a resource group from Azure Portal
4. Create a workspace
5. Register the model in container registry
6. Deploy the model as endpoint
7. Once done, delete the created resources